# Object Detection with SSD
### Here we demostrate detection on example images using SSD with PyTorch

In [1]:
import torch.nn as nn
from torch.autograd import Variable
from box_utils import*
import torch.utils.data as data
import torchvision.transforms as transforms
from torch.utils.serialization import load_lua
import PIL
from PIL import Image
from ssd import build_ssd
# from models import build_ssd as build_ssd_v1 # uncomment for older pool6 model
import torch.backends.cudnn as cudnn


## Build SSD300 in Test Phase
1. Build the architecture, specifyingsize of the input image (300),
    and number of object classes to score (21 for VOC dataset)
2. Next we load pretrained weights on the VOC 2007 dataset  

In [2]:
net = build_ssd('test', 300, 21)    # initialize SSD
net.load_weights('weights/ssd_300_VOC0712.pth')

Loading weights into state dict...
Finished!


In [7]:
net.L2Norm.weight

Parameter containing:
 13.2522
 13.2542
 13.2535
 13.2536
 13.2382
 13.2538
 13.2500
 13.2522
 13.2539
 13.2535
 13.2535
 13.2480
 13.2518
 13.2526
 13.2540
 13.2536
 13.2493
 13.2510
 13.2551
 13.2538
 13.2490
 13.2551
 13.2546
 13.2541
 13.2531
 13.2498
 13.2546
 13.2531
 13.2532
 13.2664
 13.2550
 13.2540
 13.2539
 13.2534
 13.2515
 13.2554
 13.2493
 13.2538
 13.2533
 13.2456
 13.2535
 13.2518
 13.2526
 13.2556
 13.2544
 13.2544
 13.2526
 13.2535
 13.2542
 13.2534
 13.2529
 13.2521
 13.2538
 13.2529
 13.2536
 13.2538
 13.2520
 13.2541
 13.2580
 13.2529
 13.2429
 13.2534
 13.2525
 13.2540
 13.2531
 13.2535
 13.2514
 13.2536
 13.2537
 13.2526
 13.2499
 13.2541
 13.2537
 13.2535
 13.2537
 13.2541
 13.2534
 13.2480
 13.2521
 13.2512
 13.2547
 13.2540
 13.2527
 13.2540
 13.2537
 13.2539
 13.2512
 13.2539
 13.2532
 13.2512
 13.2545
 13.2536
 13.2536
 13.2532
 13.2527
 13.2544
 13.2534
 13.2535
 13.2528
 13.2540
 13.2545
 13.2527
 13.2533
 13.2546
 13.2535
 13.2541
 13.2531
 13.2540
 13.25

## Load Image 
### Here we just load a sample image from the VOC07 dataset 

In [ ]:
# image = Image.open('./data/example.jpg').convert('RGB')   # uncomment if dataset not downloaded

from data import VOCDetection, VOCroot, base_transform, detection_collate
valset = VOCDetection(VOCroot, 'val', None, None)
img_id = 16
image = valset.pull_image(img_id)
# View the sampled input image before transform
image

## Pre-process the input.  
#### Using the torchvision package, we can create a Compose of multiple built-in transorm ops to apply 
For SSD, at test time we perform the following

In [ ]:
from data import SwapChannel
base_transform = transforms.Compose([
        transforms.Scale(net.size),                    # scale to 300 for SSD300
        transforms.CenterCrop(net.size),               # crop to a square
        transforms.ToTensor(),              
        transforms.Lambda(lambda x: x.mul(255)),       # custom lambda to scale pix by 255
        SwapChannel((2, 1, 0)),                        # custom swap color channels op
        transforms.Normalize((104,117,123), (1, 1, 1)) # subtract VOC dataset means
    ])
x = base_transform(image)

## SSD Forward Pass
### Now just wrap the image in a Variable so it is recognized by PyTorch autograd

In [ ]:
x = Variable(x.unsqueeze_(0))     # wrap tensor in Variable
y = net(x)

## Parse the Detections
Filter outputs with confidence scores lower than a threshold 
Here we choose 60% 

In [ ]:
from data import VOC_CLASSES as labels
import matplotlib.pyplot as plt
import numpy as np
top_k=10

plt.figure(figsize=(10,10))
colors = plt.cm.hsv(np.linspace(0, 1, 21)).tolist()
plt.imshow(image)  # plot the image for matplotlib
currentAxis = plt.gca()

detections = y.data
# scale each detection back up to the image
scale = torch.Tensor([image.size[0],image.size[1],image.size[0],image.size[1]])
for i in range(detections.size(1)):
    j = 0
    while detections[0,i,j,0] >= 0.6:
        score = detections[0,i,j,0]
        label_name = labels[i-1]
        display_txt = '%s: %.2f'%(label_name, score)
        pt = (detections[0,i,j,1:]*scale).cpu().numpy()
        coords = (pt[0], pt[1]), pt[2]-pt[0]+1, pt[3]-pt[1]+1
        color = colors[i]
        currentAxis.add_patch(plt.Rectangle(*coords, fill=False, edgecolor=color, linewidth=2))
        currentAxis.text(pt[0], pt[1], display_txt, bbox={'facecolor':color, 'alpha':0.5})
        j+=1

## View Results 

In [ ]:
plt.show()  # display label, score, and bounding boxes!